# Pacman in Automation

**Authors**: Corey Craddock, Braden Hogan, Justin Rentie, Wesley Turner

**Date Submitted**: 12/12/2017

**Description**: Create a game of Pacman with controls to vary levels of Pacman and Ghost intelligence and mazes.

# Introduction

For our CS 440 Final Project, we chose to implement artificial intelligence algorithms that we learned in the class over the course of this semester to run the classic arcade game Pacman in full automation. We decided this as our final project for a few reasons.

First, all four of use have deeply enjoyed learning artificial intelligence using games this semester, and we wanted our last project to reflect this. 

Second, we have already seen the AI algorithms, which are Iterative Deepening Search and Q Table Reinforcement Learning, implemented within a game setting. We were expecting implementing these algorithms for a new game to be both an obtainable goal, as well as a challenging one.

Last, in our initial research for this project, we found that implementing artificial intelligence with Pacman was a common project for computer scientists, taking the form of a rite of a passage for anyone deeply interested in artificial intelligence.

Over the course of the last few weeks, we have worked diligently to implement a game of Pacman with these algorithms. Below, we discuss our implementation for the framework of the project, the object used to save the state and players, the Iterative Deepening Search algorithm with Pacman and Ghosts, and the Reinforcement Learning algorithm with Pacman.

# Implementation

We've outlined thoughts on design and decisions that we made in implementing each of these sections here.

## Framework - Corey Craddock

Waiting for english from Corey

## Gameboard - Braden Hogan

Waiting for english from Braden

## Iterative Deepening Search - Justin Rentie

##### Overview:
It was our intention in designing our Pacman project that we would implement two different kinds of Artificial Intelligence for PacMan. We wanted to implement Iterative-Deepening Search for the first.

##### Resources:
We referenced a couple lecture notes from class in implementing IDS for Pacman. We used Lecture 05 Iterative Deepening and Other Uninformed Search Methods and 06 Python Implementation of Iterative Deepening as an example of how to implement basic IDS for searching through a tree of possible states. We also relied on examples from Assignment 02.  

_Links to these references are found [below](#References)._

##### Implementation:
We used Iterative Deepening Search to search within a given depth limit from possible actions that can be taken from pacman at any given state. If pacman locates a dot within the given depth limit the path to that dot is returned. 

The only major change that needed to be implemented was in the case of approaching a dot near any ghosts. In this situation we added a function called `fearFactor()` which will return `run` from `depthLimitedSearch()` which then in turn calls a function which removes possible moves from Pacman that could lead him towards a ghost. This results in Pacman effectively "running away" from the ghosts. 

This will go on until Pacman finds a dot within the given depth limit that is not in the direction of a ghost.

In [4]:
    # PacMan "blood hunter" AI. Copied from Ghost.
    # Helps Intelligent Move in finding the best direction to take to get to nearest dot
    def depthLimitedSearch(self, board, ghosts, closestDots, actions, takeAction, depthLimit):
        if PacMan.fearFactor(self, ghosts):
            return 'run'

        for dot in closestDots:
            if self.location == dot.location:
                return []

        if depthLimit == 0:
            return "cutoff"

        cutOffOccurred = False
        for action in PacMan.actions(self, board):
            copyBoard = copy.deepcopy(board)
            copySelf = copy.deepcopy(self)
            takeAction(copySelf, copyBoard, action)
            result = PacMan.depthLimitedSearch(copySelf, copyBoard, ghosts, closestDots, actions, takeAction,
                                              depthLimit - 1)

            if result is "cutoff":
                cutOffOccurred = True
            elif result is not "failure":
                return action
        if cutOffOccurred:
            return "cutoff"
        else:
            return "failure"

    # Causes the ghost to scan through the board, making the most intelligent shortest path decision
    def intelligentMove(self, board, ghosts, maxDepth=4):
        closestDots = PacMan.getClosestDots(self, board)

        for dot in closestDots:
            if self.location == dot.location:
                return

        for depth in range(maxDepth):
            result = PacMan.depthLimitedSearch(self, board, ghosts, closestDots, PacMan.actions, PacMan.takeAction,
                                              depth)
            if result != "cutoff" and result != "failure" and result != 'run':
                print("PacMan found intelligent move. Returning intelligentMove")
                return PacMan.takeAction(self, board, result)

        if(result == 'run'):
            PacMan.runFromGhost(self, board, ghosts,  PacMan.actions)
        else:
            PacMan.makeRandomMove(self, board)

### Example:

Example run of Pacman using Iterative-Deepening Search

In [19]:
import PlayGame as game
import PacMan as P
import Ghost as G
import Dot as d
import GameBoard as Board
import copy as copy

In [20]:
 # Really basic state to start with
state = [['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '='],
        ['|', ' ', ' ', ' ', ' ', 'G', ' ', ' ', ' ', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', ' ', '|', ' ', '|', ' ', '|', ' ', '|', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', '.', '.', '.', '.', '.', '.', '.', '.', 'P', '|'],
        ['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '=']]

In [21]:
# Start game with above state and 2 ghosts
board = Board.GameBoard(state)
# Create a Pacman object using this board
p = P.PacMan(board.pacManSpawnPt)

ghostsAvailable = [G.Ghost(board.ghostSpawnPt)]
intelligenceLevel = 3

# Initialize Q to empty array
Q = []

# Runs startGame with Q table and printing
game.startGame(board, p, ghostsAvailable, Q, intelligenceLevel, True, True)

Lives: 3 	Dots left: 8 	Location: (5, 9) 	Turn: 1 	Score: 0
|    G    |
| === === |
| | | | | |
| === === |
|........P|

Actions available: ['up', 'left']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 7 	Location: (5, 8) 	Turn: 2 	Score: 10
|    G    |
| === === |
| | | | | |
| === === |
|.......p |

Actions available: ['left', 'right']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 6 	Location: (5, 7) 	Turn: 3 	Score: 19
|    G    |
| === === |
| | | | | |
| === === |
|......p  |

Actions available: ['left', 'right']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 5 	Location: (5, 6) 	Turn: 4 	Score: 28
|         |
| ===g=== |
| | | | | |
| === === |
|.....p   |

Actions available: ['left', 'right']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 4 	Location: (5, 5) 	Turn: 5 	Score: 37
|         |
| === === |
| | |g| | |
| === === |
|....p    |

Actions available:

## Reinforcement Learning - Wesley Turner

##### Overview:
From the beginning, we knew we wanted to write a function to train a Q table for the states that Pacman would most likely move through in a game. We planned for this to be our most optimized form of a smart Pacman, as this intelligence made the most sense for our game.

##### Resources:
We utilized two different sources to implement this. The first was from a previous assignment in this course, Assignment 5. A link for this assignment page can be found below. This assignment had students implement training a Q table and testing it with the Towers of Hanoi puzzle. Although this was a good start, that puzzle is a one-person game, so our reinforcement would need to differ vastly. However, the overall structure and design for the trainQ function initially came from this assignment.

Secondly, we took inspiration for the reinforcement for Pacman from the lecture notes titled “Reinforcement Learning for Two-Player Games”. We knew we needed to augment the reinforcement for different actions that Pacman may take, and the implementation in this lecture was sufficient to start our brainstorming.

_Links to these references are found [below](#References)._

##### Implementation:
Despite these sources aiding us with design, we were still running into issues with what actions needed reinforcement. These were the actions we knew we could reinforce:

 - Change to score
 - Pacman dies
 - Pacman wins or loses
 - Pacman eats a dot

We tried a variety of combinations of these reinforcements. For the most part, these were blind tests: which combination of reinforcements would lead Pacman to have the best and fastest results? We ended up utilizing the implementation below:

In [3]:
    def trainQ(self, board, nRepetitions, learningRate, epsilonDecayFactor, ghostsAvailable, intelligenceLevel, verbose=False):
        maxGames = nRepetitions
        rho = learningRate
        epsilonDecayRate = epsilonDecayFactor
        epsilon = 1.0
        Q = {}
        scores = []

        for nGames in range(maxGames):
            if verbose:
                print("Game:", nGames, "; Starting game.")
            epsilon *= epsilonDecayRate
            step = 0
            state = copy.deepcopy(board)
            copySelf = copy.deepcopy(self)
            ghosts = []
            score = 0
            done = False

            while not done and not copySelf.gameOver(state):
                dead = False
                copyGhosts = copy.deepcopy(ghostsAvailable)
                step += 1
                copyState = copy.deepcopy(state)
                move = PacMan.epsilonGreedy(copySelf, epsilon, Q, copyState)

                _, ghosts, copySelf, stateNew, score, dead = PlayGame.runSingleTurn(step, ghosts, copyGhosts, intelligenceLevel, copySelf, copyState, score, dead, move)
                # Full return: turn, ghosts, p, board, score, dead

                #Initial value
                if PacMan.boardMoveTuple(copySelf, state, move) not in Q:
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] = 0  # initial Q value for new state,move

                if stateNew.dotsLeft < state.dotsLeft:
                    #Pacman ate a dot. Medium positive reinforcement
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] += 3
                else:
                    #Pacman did not eat a dot. Small negative reinforcement
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] += -1
                if dead:
                    #Pacman lost a life. Large negative reinforcement
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] = -10

                if step > 1:
                    Q[PacMan.boardMoveTuple(copySelf, stateOld, moveOld)] += rho * (Q[PacMan.boardMoveTuple(copySelf, state, move)] - Q[PacMan.boardMoveTuple(copySelf, stateOld, moveOld)])

                stateOld, moveOld, scoreOld = state, move, score
                state = stateNew
            if verbose:
                if copySelf.lives > 0:
                    print("Pacman Won!")
                else:
                    print("Pacman Lost!")
            scores.append(score)
        return Q, scores

This combination of reinforcements were tested to have the best results for Pacman, as it allows him to have some flexibility when choosing a state/move that is not necessarily taking the closest path to a dot (i.e. he needs to run away). We will be displaying the performance that we found in using trainQ and the Q table it creates in a section below.

### Example:

Example run of Pacman using training Q table

In [23]:
import PlayGame as game
import PacMan as P
import Ghost as G
import Dot as d
import GameBoard as Board
import copy as copy

In [24]:
 # Really basic state to start with
state = [['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '='],
        ['|', ' ', ' ', ' ', ' ', 'G', ' ', ' ', ' ', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', ' ', '|', ' ', '|', ' ', '|', ' ', '|', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', '.', '.', '.', '.', '.', '.', '.', '.', 'P', '|'],
        ['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '=']]

In [25]:
# Start game with above state and 2 ghosts
board = Board.GameBoard(state)
# Create a Pacman object using this board
p = P.PacMan(board.pacManSpawnPt)

ghostsAvailable = [G.Ghost(board.ghostSpawnPt)]
intelligenceLevel = 3

# Train Q for p
Q = []
# Trains Q table and prints each game
Q, scores = p.trainQ(board, 30, 0.5, 0.7, ghostsAvailable, intelligenceLevel, True)
print(scores)

# Runs startGame with Q table and printing
game.startGame(board, p, ghostsAvailable, Q, intelligenceLevel, False, True)

Game: 0 ; Starting game.
Pacman Lost!
Game: 1 ; Starting game.
Pacman Lost!
Game: 2 ; Starting game.
Pacman Lost!
Game: 3 ; Starting game.
Pacman Lost!
Game: 4 ; Starting game.
Pacman Lost!
Game: 5 ; Starting game.
Pacman Lost!
Game: 6 ; Starting game.
Pacman Lost!
Game: 7 ; Starting game.
Pacman Lost!
Game: 8 ; Starting game.
Pacman Won!
Game: 9 ; Starting game.
Pacman Won!
Game: 10 ; Starting game.
Pacman Won!
Game: 11 ; Starting game.
Pacman Won!
Game: 12 ; Starting game.
Pacman Won!
Game: 13 ; Starting game.
Pacman Won!
Game: 14 ; Starting game.
Pacman Won!
Game: 15 ; Starting game.
Pacman Won!
Game: 16 ; Starting game.
Pacman Won!
Game: 17 ; Starting game.
Pacman Won!
Game: 18 ; Starting game.
Pacman Won!
Game: 19 ; Starting game.
Pacman Won!
Game: 20 ; Starting game.
Pacman Won!
Game: 21 ; Starting game.
Pacman Won!
Game: 22 ; Starting game.
Pacman Won!
Game: 23 ; Starting game.
Pacman Won!
Game: 24 ; Starting game.
Pacman Won!
Game: 25 ; Starting game.
Pacman Won!
Game: 26 ; Sta

# Methods

Below are all methods and files in our project. We've decided to show who authored which section to show that we all worked as evenly as possible in implementing this project.
 
 - **Dot.py** - All methods authored by Justin Rentie
 - **GameBoard.py** - All methods authored by Braden Hogan
 - **Ghost.py**
     - init - Corey Craddock & Justin Rentie
     - actions - Corey Craddock & Braden Hogan
     - takeAction - Corey Craddock, Braden Hogan, & Justin Rentie
     - randomMove - Corey Craddock
     - takeActionShortestDistance - Wesley Turner
     - depthLimitedSearch - Wesley Turner
     - intelligentMove - Wesley Turner
 - **Pacman.py**
     - init - Corey Craddock & Justin Rentie
     - spawnPt - Corey Craddock
     - actions - Corey Craddock & Braden Hogan
     - takeAction - Corey Craddock, Braden Hogan, & Justin Rentie
     - calculateDistance - Justin Rentie
     - getClosestDot - Justin Rentie
     - directionToObj - Justin Rentie
     - fearFactor - Justin Rentie
     - depthLimitedSearch - Justin Rentie
     - intelligentMove - Justin Rentie
     - runFromGhost - Justin Rentie
     - makeRandomMove - Justin Rentie
     - gameOver - Corey Craddock
     - getLives - Corey Craddock
     - boardMoveTuple - Wesley Turner
     - useReinforcementTable - Wesley Turner
     - epsilonGreedy - Wesley Turner
     - trainQ - Wesley Turner
 - **PlayGame.py**
     - getDots - Justin Rentie
     - runSingleTurn - Wesley Turner
     - startGame - Corey Craddock & Wesley Turner
     - main - Corey Craddock & Wesley Turner

# Results

Waiting for results that Corey will gather.

# References

* [Public GitHub Repository Used for Development](https://github.com/StaticNomad/PacMan)
* [Lecture 05 - Iterative Deepening and Other Uninformed Search Methods](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/05%20Iterative%20Deepening%20and%20Other%20Uninformed%20Search%20Methods.ipynb)
* [Lecture 06 - Python Implementation of Iterative Deepening](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/06%20Python%20Implementation%20of%20Iterative%20Deepening.ipynb)
* [Lecture 15 - Reinforcement Learning for Two-Player Games](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/15%20Reinforcement%20Learning%20for%20Two-Player%20Games.ipynb) 
* [Assignemnt 02 - Iterative-Deepending Search](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/A2%20Iterative-Deepening%20Search.ipynb)
* [Assignemnt 05 - A5 Reinforcement Learning Solution to the Towers of Hanoi Puzzle](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/A5%20Reinforcement%20Learning%20Solution%20to%20Towers%20of%20Hanoi.ipynb)